In [ ]:
!pip install -U transformers
!pip install -U simpletransformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/ReviewsWithVotes (2).csv")

# fix votes nan to 0
df["votes"] = df["votes"].fillna(0)

high_quality_threshold = 0

df["imputed_quality"] = (df["votes"] > high_quality_threshold)*1


# convert boolean cols to int
df["again"] = df["again"].astype(int)
df["instructorEnjoyed"] = df["instructorEnjoyed"].astype(int)
df["instructorAgain"] = df["instructorAgain"].astype(int)

df.head()

,votes,reviewID,reviewDate,courseID,instructorID,semester,inMajorMinor,whyTake,primaryComponent,tags,...,hours,again,instructorEffectiveness,instructorEnthusiasm,instructorAccommodationLevel,instructorEnjoyed,instructorAgain,wasAuthorized,content,imputed_quality
0,3.0,5cede120-8865-40eb-b20a-1815dd9eaa5a,2022-12-28 02:16:11.988+00,MATH0118,C3177E57E2B4E380E08925E92F272599,F22,neither,Specific interest,{Projects},"{""Chill and Relaxed"",""Project Exams""}",...,1,1,10,10,10,1,1,0,Prof Malcolm-White is wonderful. Approachable ...,1
1,3.0,465f82cd-ccc0-4439-9fc7-2acbbc93bbd5,2023-01-20 13:41:48.382+00,ECON0229,7D4C6B2D9F24BDB5809D5947FC7338BC,F22,major,Elective for Major/Minor,"{Readings,Lectures,Papers}","{""Chill and Relaxed"",""Constant Reading"",""Easy ...",...,10,1,9,10,10,1,1,1,"In Econ Hist, Gregg is a phenomenal professor ...",1
2,3.0,7cd057fb-e17b-403a-8a32-91fa16d6abb7,2022-12-18 21:32:13.937+00,MATH0223,D3CD48D2B8CAD51ACB0F48C000189315,F21,major,Required for Major/Minor,"{Lectures,Homework,Exams}","{""Lots of Homework"",""Fair Grading""}",...,10,1,8,7,8,1,1,1,Professor Olinick does a good job of thoroughl...,1
3,3.0,e49d6b17-083c-4022-88c3-e76866b50313,2022-11-14 08:22:50.083+00,MATH0200,775EF6B5D466E85B040334FBC42FF7AC,F21,minor,Required for Major/Minor,{Exams},"{Fast-Paced,""Fair Grading"",""Difficult Exams""}",...,8,1,6,9,7,1,1,0,The material is pretty hard. He gives you prob...,1
4,3.0,e14841bb-4e14-4b1a-8193-a3a7432ff6ba,2022-11-08 15:09:31.648+00,PHYS0155,F16ADCB6066D31DAF05DD3D79133CD18,F22,neither,Someone recommended it,"{Exams,Homework}","{""Chill and Relaxed"",""Lots of Homework"",""Easy ...",...,3,0,8,10,8,1,0,1,Professor Glikman is very enthusiastic about t...,1


In [ ]:
def get_length(row):
    return len(row["content"].split(" "))


df["content_length"] = df.apply(get_length, axis=1)

print(df["content_length"].max())
print(df["content_length"].mean())
print(df["content_length"].median())

335
68.95376623376623
58.0


In [ ]:
val_counts = df["imputed_quality"].value_counts()
print(val_counts)
print(val_counts.max()/val_counts.sum())

1    1036
0     889
Name: imputed_quality, dtype: int64
0.5381818181818182


In [ ]:
# splitting the data into train and test dataset
X = df['content']
y = df['imputed_quality']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

train_df = pd.DataFrame(X_train)
train_df['imputed_quality'] = y_train

test_df = pd.DataFrame(X_test)
test_df['imputed_quality'] = y_test

train_df.shape, test_df.shape

((1540, 2), (385, 2))

In [ ]:
print(X_train)

125     I really enjoyed this intro class and the prof...
1616    Marion Wells is amazing and this course was on...
971     I loved this class. The content was engaging, ...
1695    For an intro-level class, he is a good profess...
1796    Pete was in the process of restructuring his g...
                              ...                        
851     This was my favorite course this semester. Pro...
701     Professor Greeley creates a fun and welcoming ...
100     Professor Gauvin is great. She clearly explain...
1106    The course was the best course I've taken in m...
1782    Hector is great. He's passionate about the top...
Name: content, Length: 1540, dtype: object


In [ ]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = train_df[(train_df['imputed_quality']==0)] 
df_minority = train_df[(train_df['imputed_quality']==1)] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= len(df_majority.index), # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# They are lot of arguments to play with
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 48,
   'eval_batch_size': 1,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 8,
   'weight_decay': 0.01,
   'learning_rate': 1e-6,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 100,
   'evaluate_during_training': True,
   'evaluate_during_training_steps': 2000,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
   'wandb_project': 'test_xlnet',
   "save_model_every_epoch" : False,
   "model_args.train_custom_parameters_only" : True,
   "model_args.custom_parameter_groups" : [
      {
          "params": ["classifier.weight"],
      },
      {
          "params": ["classifier.bias"],
      },
    ]
}

# Create a ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased', args=args) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
# Train the model
model.train_model(train_df, eval_df = test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1540 [00:00<?, ?it/s]

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

Running Epoch 6 of 8:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Running Epoch 7 of 8:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


(264,
 defaultdict(list,
             {'global_step': [33, 66, 99, 132, 165, 198, 231, 264],
              'train_loss': [0.773681640625,
               0.52569580078125,
               0.70001220703125,
               0.4930419921875,
               0.62615966796875,
               0.65771484375,
               0.62799072265625,
               0.71148681640625],
              'mcc': [0.0521799346023433,
               0.19236381649425985,
               0.23784347221951263,
               0.2975677160969849,
               0.33386214363996164,
               0.3543284521397487,
               0.3757684393135313,
               0.37560794443703627],
              'tp': [112, 150, 133, 145, 150, 155, 155, 156],
              'tn': [91, 82, 106, 106, 108, 107, 111, 110],
              'fp': [87, 96, 72, 72, 70, 71, 67, 68],
              'fn': [95, 57, 74, 62, 57, 52, 52, 51],
              'auroc': [0.52042284101395,
               0.6360527601367856,
               0.6928567551430278,


In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Running Evaluation:   0%|          | 0/385 [00:00<?, ?it/s]

Training loss,█▁
acc,▁▄▅▆▇███
auprc,▁▄▆▇████
auroc,▁▄▆▇████
eval_loss,█▅▄▃▂▁▁▁
fn,█▂▅▃▂▁▁▁
fp,▆█▂▂▂▂▁▁
global_step,▁▂▃▃▄▅▆▆▇█
lr,█▁
mcc,▁▄▅▆▇███
tn,▃▁▇▇▇▇██


In [ ]:
result

{'mcc': 0.37560794443703627,
 'tp': 156,
 'tn': 110,
 'fp': 68,
 'fn': 51,
 'auroc': 0.7563914671877545,
 'auprc': 0.7942137035907963,
 'acc': 0.6909090909090909,
 'eval_loss': 0.6043755073051948}

In [ ]:
test_content = test_df['content'].values.tolist()
predictions, raw_outputs = model.predict(test_content)
print(sklearn.metrics.classification_report(test_df['imputed_quality'], predictions))

  0%|          | 0/385 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.68      0.62      0.65       178
           1       0.70      0.75      0.72       207

    accuracy                           0.69       385
   macro avg       0.69      0.69      0.69       385
weighted avg       0.69      0.69      0.69       385



In [ ]:
print(sklearn.metrics.confusion_matrix(test_df['imputed_quality'], predictions))

[[110  68]
 [ 51 156]]


In [ ]:
print(test_df['content'])

22      Prof Collaer is an amazing instructor. She is ...
390     Ways of seeing is usually a small class. This ...
172     Professor García is such a kind, thoughtful an...
636     Eco Evo with dave Allen was a very easy course...
1719    Very real perspective into Con Bio, although a...
                              ...                        
1495    Prof Rao is a great teacher. I enjoyed his sta...
3       The material is pretty hard. He gives you prob...
515     Gerke is very chill and knowledgeable. A lot o...
88      Being almost entirely project based, 401 is qu...
319     I found this class very interesting and prof J...
Name: content, Length: 385, dtype: object


In [ ]:
test_content = [["DO NOT TAKE THIS COURSE IF YOU VALUE HAVING A LIFE. The assignments are basically impossible and not even original. Every assignment has a deadline of 1 week to write enormous amounts of code and tests for that code. It is terrible."],
                ["Prof Sanches is WONDERFUL. I cannot express that enough. She is so kind, smart, funny, passionate, and accommodating. The course material is really interesting and she is very knowledgeable. There are readings and reading responses for very class, but she is generally flexible with deadlines. I haven’t gone through a single class without laughing. She’s an incredible professor and I definitely recommend the course, although it is not necessarily light on assignments."],
                ["Ajay Verghese is a great professor - he makes things very clear and is very accommodating. The only reason I didn't love this class is because of the subject material. I took it to satisfy the poli sci methods requirement"],
                ["This class is good. It is a lot of work but if you put in the effort you can get an A. I would reccomend"],
                ["This has been one of the best classes I've taken at Middlebury. Honestly, I couldn't reccomend it more. Professor Chodrow was very understanding and I loved his 'ungraded' grading style. In this class you will learn all you need to know about machine learning, from linear regression all the way to deep learning. Chodrow is great at explaining dificult concepts. There are weekly blog posts that will take 8-10 hours a week so be prepared, but the amount you learn from them makes it worth it. This class is a must in the Comp Sci department."],
                ["This has been one of the best classes I've taken at Middlebury. Honestly, I couldn't reccomend it more. Professor Chodrow was very understanding and I loved his 'ungraded' grading style. In this class you will learn all you need to know about machine learning, from linear regression all the way to deep learning. Chodrow is great at explaining dificult concepts. There are weekly blog posts that will take 8-10 hours a week so be prepared, but the amount you learn from them makes it worth it. This class is a must in the Comp Sci department."]]
predictions1, raw_outputs = model.predict(test_content)
print(predictions1)

  0%|          | 0/385 [00:00<?, ?it/s]

[1, 1, 1, 1, 1, 0]
